One of the general use cases for our "USGS graph" is pulling together all of the various data and information assets associated with a given topical are of some kind. With the graph, we've essentially stitched together all of the functionally disconnected catalogs and inventories we have built up separately for our people and their skills, publications and datasets and the subject matters they address, and all the other pockets of information we have describing and containing our work. The process of building the graph exercises all of that, evaluating and making decisions about the relationships between things, harmonizing different content models, and putting everything together in a way that it can be queried as a whole.

One specific exercise of this concept is toward a catalog of all USGS climate science capabilities. On the one hand, this can be just about everything we do across earth system science, but we need to narrow in on those assets that are most closely connected to the topic area. This notebook explores the start to that process via a full text query across nodes containing likely content where we can initiate a search. From this, we can pull together a sub-graph based on the connections developed here.

In [1]:
import isaid_helpers
import pandas as pd

Figuring out what to query on is one part of the problem in any approach like this. From exercising things like our metadata keywords, we know that we don't have very strict alignment with controlled vocabularies or reference sources that can be exploited just yet. We've teased some of this out, making decisions about what to include and how to include it in our graph in the area of DefinedSubjectMatter and UndefinedSubjectMatter nodes.

For climate science and this exercise, I've pulled in a glossary of terms from EPA that seem like they are reasonable as a starting point.

In [2]:
epa_climate_change_glossary = pd.read_excel("https://sor.epa.gov/sor_internet/registry/termreg/searchandretrieve/glossariesandkeywordlists/search.do;jsessionid=87TsZz9rOm0k-CQjrtbfA43w1rPWKUuBf9N25Wg0XwzV9z4-yqOh!1399359231?details=&d-1342820-e=11&6578706f7274=1&glossaryName=Glossary+Climate+Change+Terms&includeInReport=")
epa_climate_change_glossary.head()

,Term,Definitions,Acronym
0,100-Year Flood Levels,Severe flood levels with a one-in-100 likeliho...,NaN
1,Abrupt Climate Change,"Sudden (on the order of decades), large change...",NaN
2,Adaptation,Adjustment or preparation of natural or human ...,NaN
3,Adaptive Capacity,The ability of a system to adjust to climate c...,NaN
4,Aerosols,Small particles or liquid droplets in the atmo...,NaN


To support this type of vector into the graph, we set up a full text index on names and descriptions across any of the nodes likely to contain reasonably useful content. We can run our search against this by sending all of our terms from the EPA glossary with OR conjunctions.

In [8]:
with isaid_helpers.graph_driver.session(database=isaid_helpers.graphdb) as session:
    results = session.run("""
    CALL db.index.fulltext.queryNodes("namesAndDescription", "%(search_list)s") YIELD node, score
    RETURN node.name AS name, labels(node)[0] AS item_type, node.url AS url, node.email AS email, node.orcid as orcid, node.doi AS doi, score
    """ % {
        "search_list": "'"+"'""' OR '".join(list(epa_climate_change_glossary.Term))+"'"
    })
    query_results = results.data()
len(query_results)

44075

We do get a reasonable result set that seems logical for what we know we have in the graph at the moment. This represents about a third of our total nodes to date retrieved simply by looking for one take on key concepts associated with climate change science.

In [9]:
df_query_results = pd.DataFrame(query_results)
df_query_results[['name','item_type']].groupby(['item_type']).agg(['count'])

,name
,count
item_type,
CreativeWork,25299
Dataset,16395
DefinedSubjectMatter,393
Event,25
Expertise,420
JobTitle,53
Journal,106
Organization,310


Some of these items are direct assets we would think about for our virtual catalog while others are starting points to network into the catalog via their relationships (e.g., we only got the people directly described in some way with these terms and we'd want to follow the graph to get everyone who has contributed).

In [10]:
df_query_results

,name,item_type,url,email,orcid,doi,score
0,Preparing for climate change: The potential co...,CreativeWork,http://www.climateimpacts.org/us-climate-asses...,None,None,None,35.249580
1,Calculation of solar radiation in mountainous ...,CreativeWork,https://doi.org/10.1016/0168-1923(87)90061-X,None,None,10.1016/0168-1923(87)90061-X,33.785244
2,Global climate change and wildlife in North Am...,CreativeWork,https://wildlife.org/technical-reviews/,None,None,None,27.945711
3,Solar radiation for National Hydrography Datas...,Dataset,"['https://doi.org/10.5066/P9OD7FAL', 'doi:10.5...",None,None,None,26.898672
4,Solar radiation for National Hydrography Datas...,Dataset,"['https://doi.org/10.5066/P9OD7FAL', 'doi:10.5...",None,None,None,26.536013
...,...,...,...,...,...,...,...
44070,Geodatabase of the available top and bottom su...,Dataset,http://water.usgs.gov/lookup/getspatial?ds40ms...,None,None,None,0.179344
44071,External quality-assurance results for the Nat...,CreativeWork,http://pubs.water.usgs.gov/wri994072,None,None,10.3133/wri994072,0.179344
44072,Geology and ore deposits of the Whitepine area...,CreativeWork,https://doi.org/10.3133/ofr56103,None,None,10.3133/ofr56103,0.166510
44073,Detrital zircon geochronology of some neoprote...,CreativeWork,https://doi.org/10.1130/2007.2431(07),None,None,10.1130/2007.2431(07),0.155390


In [ ]:
df_query_results.loc[df_query_results.email.notnull()]